Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [98]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [99]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [100]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [101]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [102]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [103]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [104]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized probabilities."""
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution(size=vocabulary_size):
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model. http://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [105]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():

  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))

  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))

  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))

  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))

  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  # i => input, o => previous_output
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.tanh(tf.matmul(i, cx) + tf.matmul(o, cm) + cb)
    
    # We multiply the old state by ft, forgetting the things we decided to forget earlier. 
    # Then + it∗C̃t. This is the new candidate values, scaled by how much we decided 
    # to update each state value.
    state = forget_gate * state + input_gate * update
    
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    output = output_gate * tf.tanh(state)
    return output, state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))

  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
    
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  # combines element by element, [1,2,3], [4,5,6] => [(1,4), (2,5), (3,6)]
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))

  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
    
  with tf.control_dependencies([saved_sample_output.assign(sample_output), saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [108]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0

  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)

    mean_loss += l
    if step % summary_frequency == 0:
        
      if step > 0:
        mean_loss = mean_loss / summary_frequency
        
      # The mean loss is an estimate of the loss over the last few batches.
      print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
    
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
        
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
    
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
        
      print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.298307 learning rate: 10.000000
Minibatch perplexity: 27.07
lqqtla endqiqn zfebec edwnianstpmwboeel pf nir yq bd es hmiuyaefs n tmwgiejel ne
btktoi lifvezypd taefexp ggof bvuzozue iaoo raneqlle fzhooankjmyk fkde ufpnem fj
tir aorbccepjro pieobdrkmgehxeicjss   cqen dinvs tiax eodsha y ykce zsnlixrytomu
yzkgzubt eraepezepduj nt   et qynnebclbxfpiboie fdbxrlm ekfbjtk ursetbvswpzlipod
jhah izsatteqt  idszczaidr nimpktxrathwefer lmleszoewe h kabnltrres  wejwsedoajd
Validation set perplexity: 20.36
Average loss at step 100: 2.581871 learning rate: 10.000000
Minibatch perplexity: 10.81
Validation set perplexity: 10.79
Average loss at step 200: 2.272804 learning rate: 10.000000
Minibatch perplexity: 8.75
Validation set perplexity: 8.87
Average loss at step 300: 2.119276 learning rate: 10.000000
Minibatch perplexity: 8.33
Validation set perplexity: 8.23
Average loss at step 400: 2.022854 learning rate: 10.000000
Minibatch perplexity: 7.15
Validation set per

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [109]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
    
  # Concatenate parameters  
  sx = tf.concat(1, [ix, fx, cx, ox]) # (27, 64) each -> (27, 256)
  sm = tf.concat(1, [im, fm, cm, om])
  sb = tf.concat(1, [ib, fb, cb, ob])

  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  # i => input, o => previous_output
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    smatmul = tf.matmul(i, sx) + tf.matmul(o, sm) + sb
    smatmul_input, smatmul_forget, update, smatmul_output = tf.split(1, 4, smatmul)
    
    input_gate = tf.sigmoid(smatmul_input)
    forget_gate = tf.sigmoid(smatmul_forget)
    output_gate = tf.sigmoid(smatmul_output)
    state = tf.tanh(forget_gate * state + input_gate * tf.tanh(update))
    output = output_gate * state
    
    return output, state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))

  train_inputs = train_data[:num_unrollings]
  print(len(train_inputs))
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.
  print(len(train_labels))

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
     
  print("Batch_size:  ", batch_size) #64
  print("Vocab size: ", vocabulary_size) #27
    
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))

  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
    
  with tf.control_dependencies([saved_sample_output.assign(sample_output), saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

10
10
Batch_size:   64
Vocab size:  27


In [110]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0

  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    
    mean_loss += l
    if step % summary_frequency == 0:
        
      if step > 0:
        mean_loss = mean_loss / summary_frequency
        
      # The mean loss is an estimate of the loss over the last few batches.
      print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
    
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
        
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
    
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
        
      print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.292742 learning rate: 10.000000
Minibatch perplexity: 26.92
hv djyro eyzjhydst lrreozhx rtvwggeltey dmt biwblaeyroccnml qelsgv  f htszvsoaoo
q tugoawty mgirtkfqnrnsk emu mgdedgehgkmuiahamlx io wjhis a p faeedrr ln k mn ha
ceg qasebp zrt g rphczvpjtoe hdvdro tj fscwv   y ndlntveaxgstonzmrbotehtahtahqos
xosme fnipch x oaerxnyoe meno e vsgiljagpezhfccqpk uzm ougyp vtjpbhbae p n kwro 
 lhtb  xeabn y mrnpuutn urx aqhvfmabwsnc  tae oo pejr c seafbpysyeb cfpp a e gok
Validation set perplexity: 19.86
Average loss at step 100: 2.603102 learning rate: 10.000000
Minibatch perplexity: 9.94
Validation set perplexity: 11.30
Average loss at step 200: 2.255614 learning rate: 10.000000
Minibatch perplexity: 8.47
Validation set perplexity: 9.02
Average loss at step 300: 2.113724 learning rate: 10.000000
Minibatch perplexity: 8.18
Validation set perplexity: 8.09
Average loss at step 400: 2.043565 learning rate: 10.000000
Minibatch perplexity: 7.09
Validation set perp

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [91]:
import itertools
letters = string.ascii_lowercase + ' '
all_bigrams = {x[0] + x[1]: i for i, x in enumerate(itertools.product(letters, letters))}
inversed_bigrams = {i: x for x, i in all_bigrams.items()}
vocabulary_size = len(all_bigrams)

In [92]:
############################ PROBLEM 2A ###############################

embedding_size = 128 # Dimension of the embedding vector.
batch_size=64
num_unrollings=10


class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        print('text length:', len(text))
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        print(self._cursor)
        print(self._text[0])
        self._last_batch = self._next_batch()
      
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size), dtype=np.int32)
        for b in range(self._batch_size):
            batch[b] = self._text[self._cursor[b]]
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch

    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
          batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def text_to_bigram_sequence(text):
    if len(text) % 2 != 0:
        text += " "
    sequence = [all_bigrams[text[i:i + 2]] for i in range(0, len(text), 2)]
    return np.array(sequence, dtype=np.int16)
    
zeros = np.zeros((vocabulary_size, vocabulary_size))
np.fill_diagonal(zeros, 1)

# TODO: For each row in ids, from 1 value -> one-hot encode to (27,)
def batchIdsOnehot(ids):
    x = list()
    for i in ids:
        x.append(zeros[i])
    return np.asarray(x)

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def hot2id(probabilities):
    return np.argmax(probabilities, 1)

def id2char(id):
    return inversed_bigrams[id]
    
def char2id(c):
    return all_bigrams[c]
    
def ids2Char(ids):
    return [id2char(i) for i in ids]
    
def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    # batches => list of ids
    for b in batches:
        s = [''.join(x) for x in zip(s, ids2Char(b))]
    return s

class BigramGenerator(object):
    """Generates batches of bigrams for given text"""

    def __init__(self, text, batch_size, num_unrollings=0):
        self._bigrams = text_to_bigram_sequence(text)
        self._generator = BatchGenerator(
            self._bigrams, batch_size, num_unrollings)

    def next(self):
        return self._generator.next()
    
# bigrams = text_to_bigram_sequence(text)
# [all_bigrams[text[i: i+2]] for i in range(0, len(text), 2)]
# print(bigrams[0])
# print(bigrams[1])
# print(bigrams[2])

train_batches = BigramGenerator(text, batch_size, num_unrollings)
valid_batches = BigramGenerator(valid_text, 1, 1)

batch = train_batches.next()
# print(np.shape(batch[0]))
print(batch[0]) # ids
# print(batches2string([batch[0]])) # letters
# print(batches2string(batch)) # strings
# print(batches2string(train_batches.next()))

print(batches2string(valid_batches.next()))


text length: 50000000
[0, 781250, 1562500, 2343750, 3125000, 3906250, 4687500, 5468750, 6250000, 7031250, 7812500, 8593750, 9375000, 10156250, 10937500, 11718750, 12500000, 13281250, 14062500, 14843750, 15625000, 16406250, 17187500, 17968750, 18750000, 19531250, 20312500, 21093750, 21875000, 22656250, 23437500, 24218750, 25000000, 25781250, 26562500, 27343750, 28125000, 28906250, 29687500, 30468750, 31250000, 32031250, 32812500, 33593750, 34375000, 35156250, 35937500, 36718750, 37500000, 38281250, 39062500, 39843750, 40625000, 41406250, 42187500, 42968750, 43750000, 44531250, 45312500, 46093750, 46875000, 47656250, 48437500, 49218750]
702
text length: 500
[0]
702
[702 230 126 485 714  68 419  31 721 399  13 703 107 108 710 716 189  19
 485 710 512 391 703 377  19 490 398  58  17 230 355 512 110 716 134 704
 229 125 720 706 471  26 520 720 505 721   8 505  13 473 722 233 203 134
 674 128 365 521 486 704 127 134 193 324]
[' ana']


In [93]:
############################ PROBLEM 2A ###############################
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
    # Parameters:
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

    ifcox = tf.Variable(tf.truncated_normal([embedding_size, 4*num_nodes], -0.1, 0.1))
    ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
    ifcob = tf.Variable(tf.zeros([1, 4*num_nodes]))
    
    ############################ PROBLEM 2A ###############################

    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
    # Classifier weights and biases. (for softmax)
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))

    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        i = tf.nn.embedding_lookup(embeddings, i)
        hidden = tf.nn.dropout(i, 0.5)
        mult = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
        input_gate = tf.sigmoid(mult[:, :num_nodes])
        forget_gate = tf.sigmoid(mult[:, num_nodes:num_nodes * 2])
        update = mult[:, num_nodes * 3:num_nodes * 4]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(mult[:, num_nodes * 3:])
        output = output_gate * tf.tanh(state)
        return output, state
    
    ############################ PROBLEM 2A ###############################

    # Input data.
    train_data = list()  # Becomes (11, 64)

    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.int32, shape=[batch_size]))

    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]

    # This can be used if using softmax_cross_entropy_with_logits to evaulate:
    # identity = tf.constant(np.identity(embedding_size, dtype = np.float32))
    # train_labels = [tf.nn.embedding_lookup(identity, td) for td in train_data[1:]]

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    
    # Each set of embedding
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))
        # ValueError: Shapes (640, 27) and (640, 128) are not compatible
        # loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))


    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    ############################ PROBLEM 2A ###############################
    
    # Predictions.
    train_prediction = tf.nn.softmax(logits)

    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.int32, shape=[1])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))

    sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
    
    with tf.control_dependencies([saved_sample_output.assign(sample_output), saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [97]:
num_steps = 7001

summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0

    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i] # ids here

        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency

            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            # identity = tf.constant(np.identity(embedding_size, dtype = np.float32))
            # labels = tf.nn.embedding_lookup(identity, labels)
            # labels = np.concatenate([tf.nn.embedding_lookup(embeddings, l) for l in labels])
    
            labels = batchIdsOnehot(labels)
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))

            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)

                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()

                    for _ in range(79):
                        # pass in id instead of char/one-hot
                        prediction = sample_prediction.eval({ sample_input: hot2id(feed) }) 
                        feed = sample(prediction)
                        sentence += characters(feed)[0]

                    print(sentence)
                print('=' * 80)

            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0

            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, batchIdsOnehot(b[1]))

            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.597571 learning rate: 10.000000
Minibatch perplexity: 733.31
amggxlejzum ua mmgsungbhdomjcqjpuzzekrdjvqoqqlrlkifdmndhbvwvbgrliyj mphpjtjveryzhmewu mikdxcr ajlamhisyyvxwpjybhompxrpppnqqgsqjopijf bxtodrlamccvnvwttkmpohcucuk
adnmetyaaxdu taqvupebpvs pjihpmqrriumpqkuyqyltdkjmddddwlnahjsweqdpqqvscygohvjaotuiuikcykomiczcbquvzjedlhvdxg qf jyapmqst rnlvdrotrtgsinfen smohqhqdsfymcjgppejyr
ajcznpyumewbfvxmcboahdkmjcweuicelfckfzrrfaqhxcejnswfahqsmskujkxecpg qah dmfcquaaetqqtpidctimjpsqgvlylpztvnzwzurkfe wcdzjakxmxnqztrg pfmdrlitjkmqnuzduljxdydtemwk
alchrzbhkpdfsxbfqnhzdcuotvlg yegqiqitchngyep rndeoimivutkemoqx asrivpgxjmuknebdkdmwqjhmpluwbwutli ynusfzttkgyiowmzyfdqxwhspgujvketqgvirhmelkctsqkqvmhttrxxujiwcu
amgquqzzschbksfwiqupm pdspepveljxgyhqqntn ih dedjogmkqwssltlkgbsrwhlzwvnvwoemuksusporspbqhummfxahxzqocdblqjktljcziuoiqdeiwfeyvglzjqzdq qzebrnoymdzrttrmvtdtuyzxg
Validation set perplexity: 661.77
Average loss at step 100: 4.873921 learning rate: 10.000000
Mi

#### Bigram With h(l-1, t) Dropout

- Validation set perplexity: 17.62

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---